## Read the csv or excel file. 
Dump the data into the mysql database.
index the column after the data is inserted.


In [22]:
import pandas as pd
df=pd.read_csv("used_bikes.csv")   #Extracting contents of CSV file into dataframe
df.head()

,bike_name,price,city,kms_driven,owner,age,power,brand
0,TVS Star City Plus Dual Tone 110cc,35000.0,Ahmedabad,17654.0,First Owner,3.0,110.0,TVS
1,Royal Enfield Classic 350cc,119900.0,Delhi,11000.0,First Owner,4.0,350.0,Royal Enfield
2,Triumph Daytona 675R,600000.0,Delhi,110.0,First Owner,8.0,675.0,Triumph
3,TVS Apache RTR 180cc,65000.0,Bangalore,16329.0,First Owner,4.0,180.0,TVS
4,Yamaha FZ S V 2.0 150cc-Ltd. Edition,80000.0,Bangalore,10000.0,First Owner,3.0,150.0,Yamaha


## Change the datatypes of the column and check the memory usage before and after the change in the data types.

In [23]:
print("Memory aefore data type change \n\n")
df.info(memory_usage="deep")  #Memory usage

Memory aefore data type change 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bike_name   149 non-null    object 
 1   price       149 non-null    float64
 2   city        149 non-null    object 
 3   kms_driven  149 non-null    float64
 4   owner       149 non-null    object 
 5   age         149 non-null    float64
 6   power       149 non-null    float64
 7   brand       149 non-null    object 
dtypes: float64(4), object(4)
memory usage: 45.0 KB


In [56]:
print("Memory after data type change \n\n")
df2=df.convert_dtypes(infer_objects=True, convert_string=True,
      convert_integer=True, convert_boolean=True, convert_floating=True)  #Converting column data types to it's best data types
df2.info(memory_usage="deep")

Memory after data type change 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   bike_name   149 non-null    string
 1   price       149 non-null    Int64 
 2   city        149 non-null    string
 3   kms_driven  149 non-null    Int64 
 4   owner       149 non-null    string
 5   age         149 non-null    Int64 
 6   power       149 non-null    Int64 
 7   brand       149 non-null    string
dtypes: Int64(4), string(4)
memory usage: 45.6 KB


## Dump the data into the mysql database.

## + 

## Index the column after the data is inserted.

In [77]:
from sqlalchemy import create_engine
import mysql.connector
import pymysql
database_config = {
    'user': 'root',
    'password': 'Mayur@2000',       #replace ***** with correct password
    'host': 'localhost',       # Change this if your MySQL server is on a different host
    'database': 'sakila',      #Change database/schema name
}
connection = mysql.connector.connect(**database_config)   #Connection is created
read_data=df2.astype(str)                                 #Read df2 and convert all datatypes to string(not necessary) but then code will need changes
source_schema_name='sakila'
source_table_name="used_bikes"
try:
                if connection:
                        
                        cursor = connection.cursor()
                        print(f"You're connected: ")
                        print(f"data load started")

                        a=""
                        for i in range(1,len(read_data.columns)+1):
                                a+=f":{i},"
                        a=a.strip(",")

                        print(f"CREATE TABLE IF NOT EXISTS {source_schema_name}.{source_table_name}")
                        
                        cursor.execute(f"""
    CREATE TABLE IF NOT EXISTS {source_table_name} (
        bike_name VARCHAR(255),
price  VARCHAR(255),
city VARCHAR(255),
kms_driven VARCHAR(255),
owner VARCHAR(255),
age VARCHAR(255),
power VARCHAR(255),
brand VARCHAR(255))
    """)
                        connection.commit()
                        print(f"Table Created {source_table_name} ")
                        
                        
                        print("Remove indexing")    #Removes the indexing(Will give error if is does not exists)
                        cursor.execute(f"""ALTER TABLE used_bikes DROP INDEX bike_index;""")
                        connection.commit()
                        print("Indexing Removed")
                        
                        
                        
                        print(f'Inserting data into table....')
                        print("Insert Query Running")
                        sql=f"""INSERT INTO {source_table_name} (bike_name, price,city,kms_driven,owner,age,power,brand)
                        VALUES (%s, %s,%s,%s,%s,%s,%s,%s)"""

                        for row in read_data.itertuples(index=False):
                            cursor.execute(sql, row)
                        connection.commit()
                        print("Record inserted succesfully")
                    
                    
                        print("Create indexing")          #Create indexing
                        cursor.execute(f"""CREATE INDEX bike_index ON used_bikes(price,age);""")
                        connection.commit()
                        print("Indexing Created")

finally:
        cursor.close()
        connection.close()

You're connected: 
data load started
CREATE TABLE IF NOT EXISTS sakila.used_bikes
Table Created used_bikes 
Remove indexing
Indexing Removed
Inserting data into table....
Insert Query Running
Record inserted succesfully
Create indexing
Indexing Created
